# Definición de problemas variacionales

Firedrake usa un lenguaje de alto nivel, UFL (https://arxiv.org/abs/1211.4047), para describir problemas variacionales. Para hacer esto, necesitamos una serie de piezas. Necesitamos una representación del dominio en el que estamos resolviendo el PDE : Firedrake usa a $\texttt{Mesh()}$ para esto (https://www.firedrakeproject.org/firedrake.html#firedrake.mesh.Mesh). Encima de esta malla, construimos $\texttt{FunctionSpaces}$ (https://www.firedrakeproject.org/firedrake.html#firedrake.functionspaceimpl.FunctionSpace) que definen el espacio en el que viven las soluciones de nuestra ecuación. Finalmente, definimos $\texttt{Functions}$ (https://www.firedrakeproject.org/firedrake.html#firedrake.function.Function) en esos espacios de funciones para contener realmente las soluciones.

## Construyendo Mallas

Firedrake puede leer mallas en formatos Gmsh , triángulo , CGNS y Exodus . Para construir una malla se usa el constructor $\texttt{Mesh()}$, pasando el nombre del archivo como argumento. El tipo de malla está determinado por la extensión del archivo, por ejemplo, si el nombre de archivo proporcionado es $\texttt{coastline.msh}$, se supone que la malla está en formato Gmsh, en cuyo caso puede construir un objeto de malla así:

In [ ]:
coastline = Mesh("coastline.msh")

Este funciona tanto en serie como en paralelo, Firedrake se encarga de descomponer la malla entre procesadores de forma transparente.

### Reordenando mallas para un mejor rendimiento 

La mayoría de los generadores de mallas producen mallas mal numeradas (con localidad de datos incorrecta) que pueden reducir el rendimiento del ensamblaje y la resolución de problemas de elementos finitos. Por defecto, entonces, Firedrake reordena las mallas de entrada para mejorar la localidad de los datos al realizar el reordenamiento inverso de Cuthill-McKee en la matriz de adyacencia de la malla de entrada. Si sabe que su malla tiene una buena numeración (quizás su generador de malla usa curvas de relleno de espacio para numerar entidades), entonces puede desactivar este reordenamiento pasando $\texttt{reorder=False}$ al constructor $\texttt{Mesh()}$ apropiado. Puede controlar el comportamiento predeterminado de Firedrake al reordenar mallas con el parámetro $\texttt{"reorder_meshes"}$. Por ejemplo, para desactivar el reordenamiento de malla globalmente:

In [ ]:
from firedrake import *
parameters["reorder_meshes"] = False

Firedrake numera los grados de libertad en un espacio funcional visitando cada celda en orden y realizando una numeración en profundidad de todos los grados de libertad en esa celda. Por lo tanto, si su malla tiene una buena numeración, los grados de libertad también la tendrán.

### Funciones de malla de utilidad

Además de ofrecer la capacidad de leer la información de la malla de un archivo, Firedrake también proporciona una serie de tipos de malla integrados para una serie de formas estándar. Los intervalos unidimensionales se pueden construir con $\texttt{IntervalMesh()}$ (https://www.firedrakeproject.org/firedrake.html#firedrake.utility_meshes.IntervalMesh); rectángulos bidimensionales con $\texttt{RectangleMesh()}$ (https://www.firedrakeproject.org/firedrake.html#firedrake.utility_meshes.RectangleMesh); y cajas tridimensionales con $\texttt{BoxMesh()}$ (https://www.firedrakeproject.org/firedrake.html#firedrake.utility_meshes.BoxMesh). También hay constructores más específicos (por ejemplo, para construir mallas cuadradas unitarias). Ver https://www.firedrakeproject.org/firedrake.html#module-firedrake.utility_meshes para detalles completos.

Además de las mallas simples descritas anteriormente, Firedrake también tiene soporte para resolver problemas en variedades sumergidas orientables . Es decir, mallas en las que las entidades se sumergen en un espacio dimensional superior. Por ejemplo, la superficie de una esfera en 3D.

Si su malla es una variedad tan inmersa, debe decirle a Firedrake que la dimensión geométrica del campo de coordenadas (que define dónde están los puntos en la malla) no es la misma que la dimensión topológica de las entidades de la malla. Esto se hace pasando un segundo argumento opcional al constructor de mallas que especifica la dimensión geométrica. Por ejemplo, para la superficie de una esfera incrustada en 3D usamos:

In [ ]:
sphere_mesh = Mesh('sphere_mesh.node', dim=3)

Firedrake proporciona mallas de utilidad para las superficies de esferas inmersas en 3D que se aproximan mediante una malla icosaédrica . Puede construir una malla de la esfera unitaria con $\texttt{UnitIcosahedralSphereMesh()}$ (https://www.firedrakeproject.org/firedrake.html#firedrake.utility_meshes.UnitIcosahedralSphereMesh), o una malla de una esfera con radio especificado usando $\texttt{IcosahedralSphereMesh()}$ (https://www.firedrakeproject.org/firedrake.html#firedrake.utility_meshes.IcosahedralSphereMesh). Las mallas se construyen refinando recursivamente un icosaedro regular , puede especificar el nivel de refinamiento pasando un valor distinto de cero $\texttt{refinement_level}$ al constructor.